In [1]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import scipy.stats as stats #conda install scikit-learn
import sklearn.linear_model as linear_model
import seaborn as sns
from sklearn.model_selection import KFold
from IPython.display import HTML, display
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 20

train = pd.read_csv('bbva_train.csv')
test =  pd.read_csv('bbva_test_incomplete.csv')

In [2]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils import check_array
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from scipy import sparse
from sklearn.preprocessing import Imputer
imputer = Imputer(strategy="median")
Xtrain = train.drop("ATTRITION", axis=1)
Ytrain = train["ATTRITION"].copy()
print(train.shape)
print(Xtrain.shape)
print(Ytrain.shape)

(56000, 53)
(56000, 52)
(56000,)


In [3]:
for i in range(Xtrain.shape[1]):
    if Xtrain.iloc[:,i].dtypes == object:
        lbl = LabelEncoder()
        lbl.fit(list(Xtrain.iloc[:,i].values) )
        Xtrain.iloc[:,i] = lbl.transform(list(Xtrain.iloc[:,i].values))

In [4]:
from sklearn.base import BaseEstimator, TransformerMixin

# Create a class to select numerical or categorical columns 
# since Scikit-Learn doesn't handle DataFrames yet
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_attribs = list(Xtrain.select_dtypes(include=[np.number]))

num_pipeline = Pipeline([
        ('selector', DataFrameSelector(num_attribs)),
        ('imputer', Imputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

In [6]:
Xtrain_final = num_pipeline.fit_transform(Xtrain)
Xtrain_final

array([[ 0.13025474,  0.        ,  0.43681264, ...,  0.9289127 ,
         0.93758654,  0.94426493],
       [ 0.6248649 ,  0.        ,  0.43681264, ...,  0.9289127 ,
         0.93758654,  0.94426493],
       [ 1.51244781,  0.        ,  0.43681264, ...,  0.9289127 ,
         0.93758654,  0.94426493],
       ...,
       [-0.83885921,  0.        ,  0.43681264, ..., -1.07652742,
         0.93758654,  0.94426493],
       [ 0.89457375,  0.        , -2.28931104, ..., -1.07652742,
        -1.06656821, -1.05902482],
       [ 0.81266269,  0.        ,  0.43681264, ...,  0.9289127 ,
        -1.06656821, -1.05902482]])

In [7]:
Ynew = np.zeros((Ytrain.shape[0], 1))
for i in range(0,Ytrain.shape[0]):
    Ynew[i][0] = Ytrain[i]

In [8]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE  

print('Balance',Counter(Ytrain))

from imblearn.over_sampling import SMOTE, ADASYN
#Banlancea ADASYN
X_res_o, y_res_o = ADASYN().fit_sample(Xtrain_final,Ytrain)
print('Balance',Counter(y_res_o))

Balance Counter({0: 47342, 1: 8658})
Balance Counter({0: 47342, 1: 46409})


In [9]:
Xtrain_final = X_res_o
Ynew = np.zeros((y_res_o.shape[0], 1))
for i in range(0,y_res_o.shape[0]):
    Ynew[i][0] = y_res_o[i]
    
print(Xtrain_final.shape)
print(Ynew.shape)
from sklearn.model_selection import train_test_split
import random
#Stratified split according to Ynew
X, X_test, Y, Y_test = train_test_split(Xtrain_final, Ynew, test_size=0.13, random_state=random.randrange(50), stratify=Ynew)
Y_test1 = Y_test.T[0]

(93751, 52)
(93751, 1)


In [10]:
print(Y.shape)

Y = Y.T[0]
Y_test = Y_test.T[0]
print(Y)

Y = np.array([Y, abs(-(Y-1))]).T  # The model currently needs one column for each class
Y_test = np.array([Y_test, abs(-(Y_test-1))]).T # The model currently needs one column for each class
print(Y.shape)
print(Y_test.shape)
print(Y_test1.shape)


(81563, 1)
[1. 1. 1. ... 1. 0. 0.]
(81563, 2)
(12188, 2)
(12188,)


Preprocesando Test BBVA

In [11]:
from sklearn.pipeline import FeatureUnion

for i in range(test.shape[1]):
    if test.iloc[:,i].dtypes == object:
        lbl = LabelEncoder()
        lbl.fit(list(test.iloc[:,i].values) )
        test.iloc[:,i] = lbl.transform(list(test.iloc[:,i].values))

num_attribs = list(test.select_dtypes(include=[np.number]))
        
num_pipeline = Pipeline([
        ('selector', DataFrameSelector(num_attribs)),
        ('imputer', Imputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

test_final = num_pipeline.fit_transform(test)
test_final

array([[-1.32252252,  0.        ,  0.44268259, ...,  0.94196664,
         0.9520074 ,  0.95337158],
       [-0.03600456,  0.        ,  0.44268259, ..., -1.06160872,
        -1.05041201, -1.04890897],
       [ 0.36570994,  0.        , -2.25895486, ..., -1.06160872,
        -1.05041201, -1.04890897],
       ...,
       [-0.98343324,  0.        ,  0.44268259, ...,  0.94196664,
         0.9520074 ,  0.95337158],
       [ 0.4329702 ,  0.        ,  0.44268259, ..., -1.06160872,
        -1.05041201, -1.04890897],
       [ 1.16892984,  0.        ,  0.44268259, ..., -1.06160872,
        -1.05041201, -1.04890897]])

In [16]:
# Parameters
learning_rate = 0.001
training_epochs = 500
batch_size = 100
display_step = 10


# Network Parameters
n_hidden_1 = 26 # 1st layer number of features
n_hidden_2 = 26 # 2nd layer number of features
n_input = 52 # Number of feature
n_classes = 2 # Number of classes to predict (2 classes TRUE, FALSE)


# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer (sigmoid works better that softmax)
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initializing the variables
init = tf.global_variables_initializer()

In [17]:
Xtrain_final
Ynew1 = np.zeros((Ytrain.shape[0], 1))
for i in range(0,Ytrain.shape[0]):
    Ynew1[i][0] = Ytrain[i]

In [18]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(len(X)/batch_size)
        X_batches = np.array_split(X, total_batch)
        Y_batches = np.array_split(Y, total_batch)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = X_batches[i], Y_batches[i]
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: X_test, y: Y_test}))
    #Matriz de confusion
    global result 
    result = tf.argmax(pred*(-1), 1).eval({x: X_test, y: Y_test})
    #Predicionedo Test BBVA
    BBVA = tf.argmax(pred*(-1), 1).eval({x: test_final})
    print("BBVA: ",BBVA.shape)
    #for i in result:
        #print(i)
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import precision_score
    from sklearn.metrics import f1_score
    from sklearn.metrics import recall_score
    print(confusion_matrix(Y_test1, result))
    print("Precision: ",precision_score(Y_test1, result, average='binary'))
    print("Recall: ",recall_score(Y_test1, result, average='binary'))
    print("F1: ",f1_score(Y_test1, result, average='binary'))
    
    
  

Epoch: 0001 cost= 8.228005796
Epoch: 0011 cost= 0.553749172
Epoch: 0021 cost= 0.446887085
Epoch: 0031 cost= 0.397084051
Epoch: 0041 cost= 0.370123020
Epoch: 0051 cost= 0.351989058
Epoch: 0061 cost= 0.339252506
Epoch: 0071 cost= 0.329908343
Epoch: 0081 cost= 0.323169813
Epoch: 0091 cost= 0.317440343
Epoch: 0101 cost= 0.312522847
Epoch: 0111 cost= 0.309207973
Epoch: 0121 cost= 0.306068566
Epoch: 0131 cost= 0.303110146
Epoch: 0141 cost= 0.300929818
Epoch: 0151 cost= 0.299234772
Epoch: 0161 cost= 0.297109619
Epoch: 0171 cost= 0.295336082
Epoch: 0181 cost= 0.293936739
Epoch: 0191 cost= 0.292245905
Epoch: 0201 cost= 0.291221948
Epoch: 0211 cost= 0.290552964
Epoch: 0221 cost= 0.289528579
Epoch: 0231 cost= 0.288188019
Epoch: 0241 cost= 0.287642259
Epoch: 0251 cost= 0.286285146
Epoch: 0261 cost= 0.285709655
Epoch: 0271 cost= 0.285169246
Epoch: 0281 cost= 0.284242019
Epoch: 0291 cost= 0.283693437
Epoch: 0301 cost= 0.282720765
Epoch: 0311 cost= 0.282170620
Epoch: 0321 cost= 0.281761121
Epoch: 033

In [19]:
unos = 0
for i in BBVA:
    if (i == 1):
        unos += 1
    print(i)
print("Predijo: ",unos)

0
1
0
0
1
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
1
0
0
0
0
0
1
0
0
0
0
0
0
1
1
0
0
0
1
0
0
0
1
0
0
0
1
1
0
0
0
0
1
0
0
1
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
1
1
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
0
0
1
1
0
0
0
0
0
0
0
1
1
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
1
0
1
0
0
0
0
0
1
0
1
0
0
0
0
0
1
0
0
0
0
1
0
0
0
0
1
0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0


0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
1
1
0
0
1
1
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
1
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
1
0
1
1
0
0
0
0
0
0
1
0
1
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
1
1
0
0
0
0
1
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
1
0
0
0
1
1
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
1
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
1
0
0
1
0
1
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
0
1
0
0
1
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
1
1
0
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
1
1
0
0
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
1
1
1
0
0


1
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
1
0
1
0
1
0
0
0
1
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
0
0
1
0
0
1
0
0
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
1
1
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
1
0
0
0
0
0
1
0
0
1
0
0
0
0
1
0
0
0
1
0
0
1
1
0
0
0
0
1
1
1
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
1
0
0
0
0
0
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [20]:
test_ids = test["ID_CORRELATIVO"].copy()
submission = pd.DataFrame({
    "ID_CORRELATIVO": test_ids,
    "ATTRITION": BBVA
   
})

submission.to_csv('balanceado_26.csv', index=False)
        
submission

,ID_CORRELATIVO,ATTRITION
0,12174,0
1,49090,1
2,60617,0
3,24584,0
4,70300,1
5,42687,0
6,57527,0
7,70786,0
8,60622,0
9,9447,0


In [ ]:
Predijo:  3912